In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


In [2]:
print(tf.__version__)

2.17.0


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/insurance.csv")
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
insurance.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [6]:
insurance.isnull().sum() , len(insurance)

(age         0
 sex         0
 bmi         0
 children    0
 smoker      0
 region      0
 charges     0
 dtype: int64,
 1338)

In [7]:
insurance.region.value_counts()

region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64

In [8]:
insurance_one_hot = pd.get_dummies(insurance, columns=['sex', 'children', 'smoker', 'region'], dtype=float)

In [28]:
from sklearn.model_selection import train_test_split

X = insurance_one_hot.drop('charges', axis=1)
y = insurance_one_hot.charges

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1070, 16), (1070,), (268, 16), (268,))

In [10]:
# def convert_to_tensor_type(data):
#     return tf.convert_to_tensor(data, dtype=tf.float32)
# X_train = convert_to_tensor_type(X_train)
# y_train = convert_to_tensor_type(y_train)
# X_test = convert_to_tensor_type(X_test) 
# y_test = convert_to_tensor_type(y_test)

In [29]:
X_train.shape

(1070, 16)

In [30]:
tf.random.set_seed(42)

model_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(16,)), 
    tf.keras.layers.Dense(1)
], name='Model_1')

model_1.compile(loss=tf.keras.losses.mae, 
               optimizer=tf.keras.optimizers.SGD(), 
               metrics=[
                   tf.keras.metrics.MeanAbsoluteError(name='mae'),
                   tf.keras.metrics.MeanSquaredError(name='mse')
               ])

model_1.summary()

model_1.fit(X_train, y_train, epochs=100, verbose=0)

Model: "Model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               1700      
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1801 (7.04 KB)
Trainable params: 1801 (7.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
model_1.evaluate(X_test, y_test)

9/9 [==============================] - 0s 3ms/step - loss: 8984.4033 - mae: 8984.4033 - mse: 113433344.0000


[8984.4033203125, 8984.4033203125, 113433344.0]

In [40]:
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=(16,)), 
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_2.compile(loss=tf.keras.losses.mae, 
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
               metrics=[
                   'mae',
                   'mse'
               ])

# print(model_2.summary())

model_2.fit(X_train, y_train, epochs=100, verbose=0)
model_2.evaluate(X_test, y_test)

9/9 [==============================] - 0s 2ms/step - loss: 1857.3398 - mae: 1857.3398 - mse: 21297334.0000


[1857.33984375, 1857.33984375, 21297334.0]

In [19]:
y_pred_1 = model_1.predict(X_test)

9/9 [==============================] - 0s 2ms/step


## Early Stopping Callback (Very important concept)

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

## Normalization and standardization 

https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization